### Prepare application by running all cells below

In [1]:
import os
import ipywidgets as widgets
from IPython.display import display

from comet_ml import API
import pandas as pd

from ift6758.client import ServingClient
from ift6758.game_client import GameClient

#### Get environment variables and set up the clients

In [2]:
COMET_API_KEY = os.environ.get("COMET_API_KEY")
IP = os.environ.get("SERVING", "serving")
PORT = os.environ.get("PORT_SERVING", "2000")
data_unprocessed = pd.DataFrame()
data_processed = pd.DataFrame()
previous_game_id = None

In [3]:
api = API(api_key=COMET_API_KEY)
game_client = GameClient()
serving_client = ServingClient(ip=IP, port=PORT)

#### Create all dropdowns and buttons

In [4]:
workspace_value = os.environ.get("COMET_DEFAUTL_MODEL_WORKSPACE", 'jaihon')
workspace_dropdown = widgets.Dropdown(
    options=[workspace_value],
    value=workspace_value,
    description='Workspace:',
    disabled=False,
)

models_options = api.get_registry_model_names(workspace_dropdown.value)[1:]
model_dropdown = widgets.Dropdown(
    options=models_options,
    value=models_options[2],
    description='Model:',
    disabled=False,

)

version_options = api.get_registry_model_versions(workspace_dropdown.value, model_dropdown.value)
version_input = widgets.Text(
    placeholder='Version model',
    description='Version:',
    disabled=False,

)

download_button = widgets.Button(
    description='Download',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Downloads the model',
    icon='download', # (FontAwesome names without the `fa-` prefix)
)
serving_client.download_registry_model(workspace_dropdown.value, model_dropdown.value, version_input.value)
previous_model = model_dropdown.value

In [5]:
models_widgets = widgets.HBox([workspace_dropdown, model_dropdown, version_input, download_button])

In [6]:
game_id_input = versions = widgets.Text(
    placeholder='game ID',
    description='Game ID:',
    disabled=False,
)
upload_game_info = widgets.Button(
    description='Update Predictions',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Fetch new game information and update predictions',
    icon='refresh', # (FontAwesome names without the `fa-` prefix)
)

In [7]:
game_widgets = widgets.HBox([game_id_input, upload_game_info])

#### Link buttons with client's function

In [8]:
def update_model(sender):
    workspace = workspace_dropdown.value
    model = model_dropdown.value
    version = version_input.value
    serving_client.download_registry_model(workspace, model, version)
download_button.on_click(update_model)

In [9]:
def update_predictions(sender):
    global data_unprocessed
    global data_processed
    global previous_model
    global previous_game_id
    game_id = game_id_input.value
    new_data = game_client.ping_game(game_id)

    if previous_game_id != game_id:
        previous_game_id = game_id
        data_unprocessed = pd.DataFrame()
        data_processed = pd.DataFrame()

        if len(new_data)!=0:
            pred = serving_client.predict(new_data)
            data_unprocessed=data_unprocessed.append(new_data)
            data_unprocessed.reset_index(drop=True, inplace=True)
            data_processed = data_processed.append(pred)
            data_processed.reset_index(drop=True, inplace=True)

    else:

        if len(new_data)!=0 and previous_model == serving_client.current_model:
            pred = serving_client.predict(new_data)
            data_unprocessed=data_unprocessed.append(new_data)
            data_unprocessed.reset_index(drop=True, inplace=True)
            data_processed = data_processed.append(pred)
            data_processed.reset_index(drop=True, inplace=True)

        if previous_model != serving_client.current_model:
            previous_model = serving_client.current_model
            data_unprocessed = data_unprocessed.append(new_data)
            data_unprocessed.reset_index(drop=True, inplace=True)
            pred = serving_client.predict(data_unprocessed)
            data_processed = pred

upload_game_info.on_click(update_predictions)

In [10]:
def show_predictions(b):
    with output_pred:
        output_pred.clear_output()
        if game_id_input.value != '' and len(data_unprocessed)!=0:
            game_id = game_id_input.value
            
            # get the information to print    
            home = game_client.home # home team
            away = game_client.away # away team
            pred_home = data_processed[data_unprocessed[('home',)]==1]['prediction'].astype(float).sum()
            pred_away = data_processed[data_unprocessed[('away',)]==1]['prediction'].astype(float).sum()
            goals_home = data_unprocessed[data_unprocessed[('home',)]==1]['is_goal'].astype(int).sum()
            goals_away = data_unprocessed[data_unprocessed[('away',)]==1]['is_goal'].astype(int).sum()
            info = [['', f"Period:{game_client.period}", f"Time Left: "+game_client.time_left],
                    ['', 'Home', 'Away'], 
                    ['Teams', home, away],
                    ['Expected Goals', f"{pred_home:.4f}", f"{pred_away:.4f}"],
                    ['Actual Goals', f"{goals_home}", f"{goals_away}"]
                   ]

            # print everything
            print(game_client.dateTime)
            print('Game ID: {} {} (Home) {} (Away)'.format(str(game_id)[-4:], home, away))
            for row in info:
                print('\t{: <20} {: <20} {: <20}'.format(*row))
            
            print()
            print(f"Predictions were done with these data using {serving_client.current_model} model:")
            display(data_processed)
        elif len(data_processed)==0 and game_id_input!='':
            print("Game data empty. Either the game does not exist or it hasn't happened yet.")
            
output_pred = widgets.Output()
upload_game_info.on_click(show_predictions)

In [11]:
all_widgets = widgets.VBox([models_widgets,game_widgets, output_pred])

# Run application


## How to use the app
1. Select a workspace, model and enter a version for the model.
2. Click on the download button to change the model to the selected one.
    
**Note: Even if the selected options for the model is changed, the actual model used for prediction will not change as long as you don't press download.**

3. Enter a Game ID and click on update predictions.

If you want to evaluate the predictions on a different model while on the same game, just download the model and then press on update predictions without touching the Game ID. It will predict the expected goals with the new model on the same data plus the new data if there is.

In [12]:
display(all_widgets)